In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.

In [ ]:
# import libraries
try:
%tensorflow_version only exists in Colab.
!pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.9/440.9 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 23.1.4 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.12.0-dev20230108


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv -O train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv -O valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-01-08 19:15:08--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.06s   

2023-01-08 19:15:08 (6.17 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-01-08 19:15:08--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
# Load the dataset
train_df = pd.read_csv(train_file_path, sep="\t", names=["type", "text"])
test_df = pd.read_csv(test_file_path, sep="\t", names=["type", "text"])

train_df.dropna()
test_df.dropna()

print(train_df.shape)
print(test_df.shape)

train_df.head()

(4179, 2)
(1392, 2)


,type,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
# Apply encoding to labels
str_to_int = {"ham": 0, "spam": 1}
int_to_str = {0: "ham", 1: "spam"}

train_df["type"] = train_df["type"].replace(str_to_int)
test_df["type"] = test_df["type"].replace(str_to_int)

train_df.head()

,type,text
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
# Shuffle the dataset
BUFFER_SIZE = 100
BATCH_SIZE = 64

train_ds = (
    tf.data.Dataset.from_tensor_slices(
        (
            train_df["text"].values, train_df["type"].values
        )
    )
)

test_ds = (
    tf.data.Dataset.from_tensor_slices(
        (
            test_df["text"].values, test_df["type"].values
        )
    )
)

print(test_ds.element_spec)

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))


In [ ]:
# Use the TextVectorization layer
VOCAB_SIZE = 1000
encoder = keras.layers.TextVectorization(
    output_mode='int',
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_ds.map(lambda text, label: text))

In [ ]:
#Create the model
model = keras.Sequential([
  encoder,
  keras.layers.Embedding(
      input_dim=len(encoder.get_vocabulary()),
      output_dim=64,
      mask_zero=True
  ),
  keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
  keras.layers.Bidirectional(keras.layers.LSTM(32)),
  keras.layers.Dense(64, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 64)          64000     
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

In [ ]:
# Compile and train the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_ds, epochs=10,
                    validation_data=test_ds,
                    validation_steps=30)

Epoch 1/10
66/66 [==============================] - ETA: 0s - loss: 0.6622 - accuracy: 0.8660

66/66 [==============================] - 47s 324ms/step - loss: 0.6622 - accuracy: 0.8660 - val_loss: 0.6034 - val_accuracy: 0.8657
Epoch 2/10
66/66 [==============================] - 13s 202ms/step - loss: 0.5269 - accuracy: 0.8660
Epoch 3/10
66/66 [==============================] - 13s 203ms/step - loss: 0.4012 - accuracy: 0.8660
Epoch 4/10
66/66 [==============================] - 13s 190ms/step - loss: 0.2361 - accuracy: 0.8696
Epoch 5/10
66/66 [==============================] - 13s 198ms/step - loss: 0.1294 - accuracy: 0.9610
Epoch 6/10
66/66 [==============================] - 14s 207ms/step - loss: 0.0867 - accuracy: 0.9813
Epoch 7/10
66/66 [==============================] - 14s 210ms/step - loss: 0.0693 - accuracy: 0.9835
Epoch 8/10
66/66 [==============================] - 14s 209ms/step - loss: 0.0542 - accuracy: 0.9871
Epoch 9/10
66/66 [==============================] - 14s 209ms/step - loss: 0.0474 - accuracy: 0.9895
Epoch 10/10
66/66 [==============================] - 14s 215

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

22/22 [==============================] - 1s 58ms/step - loss: 0.0598 - accuracy: 0.9856
Test Loss: 0.059840310364961624
Test Accuracy: 0.9856321811676025


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = model.predict(np.array([pred_text]))[0][0]
  type_pred = "ham" if prediction < 0 else "spam"
  print(prediction)
  return [prediction, type_pred]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 5s 5s/step
-5.308525
[-5.308525, 'ham']


In [ ]:
 Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

SyntaxError: ignored